In [207]:
from urllib import request as urllib2
from bs4 import BeautifulSoup as soup
import pandas as pd
import numpy as np
import time
import re

In [208]:
#Add required match url and player list to consider if any.
url = 'https://www.espncricinfo.com/series/19245/game/1183512/australia-women-vs-sri-lanka-women-3rd-t20i-sl-w-in-australia-2019-20'
#consider_players = "Ram Dayal, Parvez Rasool (c), Mohammed Mudhasir, Shubham Khajuria, Waseem Raza, Ahmed Banday, Umar Nazir Mir, Shubham Pundir, Jatin Wadhwan, Fazil Rashid (wk), Qamran Iqbal, Ashok Menaria, Aniket Choudhary, Arjit Gupta, Abhimanyu Lamba, Chetan Bist (wk), Manender Narender Singh, K Khaleel Ahmed, Mahipal Lomror (c), Rahul Chahar, Salman Khan, Tajinder Singh"

consider_players = ""

report_name = 'Aus W vs SL W Oct 02 2019.csv'

'''missing_players = {'Shakib Al Hasan': 'http://www.espncricinfo.com/bangladesh/content/player/56143.html', 
                  'Sunil Narine': 'http://www.espncricinfo.com/westindies/content/player/230558.html',
                  'Hayden Walsh': 'http://www.espncricinfo.com/usa/content/player/443263.html'}'''
missing_players ={}

In [209]:
int_countries = "AUS Women, SL Women, WI Women, sa, bdesh, india, pakistan, sri Lanka, bangladesh, afghanistan, south africa, new zealand, australia, england, west indies, nepal, zimbabwe, kenya, ireland, singapore, kuwaith, uae, canada, qatar, malaysia, denmark"

In [210]:
#add useragent to access cricinfo
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 

In [211]:
# Get match details
request=urllib2.Request(url,None,headers)
response = urllib2.urlopen(request)
data = response.read()
page_soup = soup(data, 'html.parser')

#Get HTML related to player list
players_list_html = (page_soup.find('article', {'class': 'boxscore-tabs sub-module-mobile-combine-split combined squad'})
                .findAll('td', {'class': 'name'}))


In [212]:
#Validate if player is required
def validate_player(player):
    result = True
    if consider_players != "":   
        result = False
        player = player.split(" ")
        for item in player:
            if item in consider_players:            
                result = True
    return result

In [213]:
players_details = pd.DataFrame(columns={'name', 'link'})
for item in players_list_html:
    player_name = item.find('a').contents[0]
    player_link = item.find('a')['href']
    if player_name and player_link:
        player_name = player_name.replace('(c)', "").replace('†', "").strip()
        if validate_player(player_name):
            player_details = pd.DataFrame({"name": [player_name], "link": [player_link]})
            players_details = players_details.append(player_details, ignore_index = True)

C:\Users\nithin.prakash\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [214]:
#Add missing Players
c = []
for item in missing_players:
    player_details = pd.DataFrame({"name": [item], "link": [missing_players[item]]})
    players_details = players_details.append(player_details, ignore_index = True)

In [215]:
'''#c = ['Bat & Bowl', 'Team', 'Opposition', 'Ground', 'Match Date', 'Scorecard']
temp = get_player_details('http://www.espncricinfo.com/ci/content/player/275447.html')
print(temp)
print(get_player_stats(temp))
len(c)'''

"#c = ['Bat & Bowl', 'Team', 'Opposition', 'Ground', 'Match Date', 'Scorecard']\ntemp = get_player_details('http://www.espncricinfo.com/ci/content/player/275447.html')\nprint(temp)\nprint(get_player_stats(temp))\nlen(c)"

In [216]:
def get_player_details(player_url):
    #get details from cricinfo
    request=urllib2.Request(player_url,None,headers)
    response = urllib2.urlopen(request)
    data = response.read()
    page_soup = soup(data, 'html.parser')
    
    c = []
    #Search for previous match details
    page_tables = page_soup.find_all('table', {'class': 'engineTable'})
    
    #Extract previous match details to dataframe
    recent_match_table = ""
    for table in page_tables:
        is_match_stats = table.find('th',{'title': 'opposition'})
        if is_match_stats:
            recent_match_table = table
    if recent_match_table != "":        
        for item in recent_match_table.find_all('th'):
            c.append(item.contents[0])
        c.append('is_international')
        previous_match_details = pd.DataFrame(columns=c)
        for item in recent_match_table.find_all('tr'):
            r = item.find_all('td')
            if len(r) == len(c) - 1:
                is_int = r[2].contents[0][2:].replace('Women', '').strip().lower() in int_countries
                data = {c[0]: [r[0].contents[0]], c[1]: [r[1].contents[0]], c[2]: [r[2].contents[0]], 
                                c[3]: [r[3].find('a').contents[0]], c[4]: [r[4].contents[0]], 
                        c[5]: [r[5].contents[0].contents[0]],
                       c[6]: [is_int]}
                row = pd.DataFrame(data=data)
                previous_match_details = previous_match_details.append(row, ignore_index=True)
        
        return previous_match_details
    else:
        return pd.DataFrame()

In [217]:
def get_bat_points(score, mode='odi'):
    runs = int(score.replace("*", "").strip())
    points = runs * 1
    if(runs > 100):
        points = runs + 20
    elif(runs > 50):
        points = runs + 10
    if(mode == 't20' and runs > 30):
        points = points + 10
    return points * 1.2
def get_bowl_points(score):
    wickets = int(score.split(sep='/')[0].strip())
    points = wickets * 20
    if wickets > 2:
        points = points + ((wickets - 2) * 10)
    return points + 5
def get_catch_points(score):
    score = score.split(sep='/')
    catch = int(score[0].replace('c','').strip())
    stump = int(score[1].replace('s','').strip())
    return catch * 10 + stump * 15

In [218]:
def get_odi_points(stats):
    bat_points = 0
    bowl_points = 0
    catch_points = 0
    stats = stats.split(sep=',')
    for stat in stats:
        if 'c' in stat:
            catch_points = get_catch_points(stat)
        elif '/' in stat:
            bowl_points = get_bowl_points(stat)
        elif '-' not in stat:
            bat_points = get_bat_points(stat)
    return bat_points, bowl_points, catch_points

def get_t20_points(stats):
    bat_points = 0
    bowl_points = 0
    catch_points = 0
    stats = stats.split(sep=',')
    for stat in stats:
        if 'c' in stat:
            catch_points = get_catch_points(stat)
        elif '/' in stat:
            bowl_points = get_bowl_points(stat)
        elif '-' not in stat:
            bat_points = get_bat_points(stat, 't20')
    return bat_points, bowl_points, catch_points

def get_test_points(stats):
    bat_points = 0
    bowl_points = 0
    catch_points = 0
    bat_ings = 0
    bowl_ings = 0
    catch_ings = 0
    
    stats = stats.split(sep=',')
    for stat in stats:
        if 'c' in stat:
            catch_ings += 1
            catch_points += get_catch_points(stat)
            catch_points = catch_points / catch_ings              
        elif '/' in stat:
            bowl_ings += 1
            bowl_points += get_bowl_points(stat)
            bowl_points = bowl_points / bowl_ings
        elif '-' not in stat:
            bat_ings += 1
            bat_points += get_bat_points(stat)
            bat_points = bat_points / bat_ings
    return bat_points, bowl_points, catch_points

In [228]:
def get_player_stats(df):
    bat_points_list = []
    bowl_points_list = []
    catch_points_list = []
    for index, match in df.iterrows():
        bat_points = 0
        bowl_points = 0 
        catch_points = 0
        if "ODI" in match['Scorecard']:
            bat_points, bowl_points, catch_points = get_odi_points(match['Bat & Bowl'])
        elif "T20" in match['Scorecard']:
            bat_points, bowl_points, catch_points = get_t20_points(match['Bat & Bowl'])
        elif "Test" or 'FC' in match['Scorecard']:
            bat_points, bowl_points, catch_points = get_test_points(match['Bat & Bowl'])
        else:
            bat_points, bowl_points, catch_points = get_test_points(match['Bat & Bowl'])
        if match['Opposition'][2:].strip() in int_countries:
            bat_points, bowl_points = bat_points * 1.2, bowl_points * 1.2
            
        bat_points_list.append(bat_points)
        bowl_points_list.append(bowl_points)
        catch_points_list.append(catch_points)
    df['bat_points'] = bat_points_list
    df['bowl_points'] = bowl_points_list
    df['catch_points'] = catch_points_list
    df['total_points'] = df['bat_points'] + df['bowl_points'] + df['catch_points']
    return df

In [ ]:
players_stat = {}
for index, item in players_details.iterrows():
    time.sleep(2)
    print(item['link'])
    player_detail = get_player_details(item['link'])
    player_stat = get_player_stats(player_detail)
    
    players_stat[item['name']] = player_stat

http://www.espncricinfo.com/ci/content/player/275447.html
http://www.espncricinfo.com/ci/content/player/381217.html
http://www.espncricinfo.com/ci/content/player/858809.html
http://www.espncricinfo.com/ci/content/player/546629.html
http://www.espncricinfo.com/ci/content/player/268889.html
http://www.espncricinfo.com/ci/content/player/275486.html
http://www.espncricinfo.com/ci/content/player/374936.html
http://www.espncricinfo.com/ci/content/player/263473.html
http://www.espncricinfo.com/ci/content/player/329336.html
http://www.espncricinfo.com/ci/content/player/381258.html
http://www.espncricinfo.com/ci/content/player/275487.html
http://www.espncricinfo.com/ci/content/player/420314.html
http://www.espncricinfo.com/ci/content/player/1071711.html
http://www.espncricinfo.com/ci/content/player/946057.html


In [ ]:
len(players_stat)
players_stat

In [ ]:
def cal_weighted_points(points, coef):
    if coef < 90:
        return points
    
    return points * (1-(coef-90) * 0.003)

In [ ]:
p_df = pd.DataFrame(columns=['player', 'weight_points','one_match_per', 'total_points', 'coef_var', 'int_matches', 
                                             'int_points', 'bat_points', 'bowl_points','catch_points'])
p = players_stat
for i in players_stat:
    try:
        df = p[i]
        coef = (df['total_points'].std() / df['total_points'].mean()) * 100
        weighted_points = cal_weighted_points(df['total_points'].mean(), coef)    
        one_match_per = (df['total_points'].nlargest(2).sum() / df['total_points'].sum()) * 100
        int_match_count = df['is_international'].sum()
        int_match_points = df[df['is_international']]['total_points'].sum() / (int_match_count + 0.001)
        row = pd.DataFrame({
                            'player': [i], 
                            'weight_points': [weighted_points],
                            'one_match_per': [one_match_per],
                            'total_points': [df['total_points'].mean()],
                            'coef_var': [coef],
                            'int_matches':[int_match_count],
                            'int_points': [int_match_points],
                            'bat_points': [df['bat_points'].mean()],
                            'bowl_points': [df['bowl_points'].mean()],
                            'catch_points': [df['catch_points'].mean()],
                           })
        p_df = p_df.append(row, ignore_index=True)
    except:
        print(df)
p_df = p_df.sort_values(by=['weight_points'], ascending=False)
consist_players = p_df[p_df['one_match_per'] < 40]
inconsist_players = p_df.loc[(p_df['one_match_per'] > 39.99) & (p_df['one_match_per'] < 50.00001)]
extremly_inconsistant_players = p_df[p_df['one_match_per'] > 50]

In [ ]:
consist_players.to_csv('consist_'+ report_name)
consist_players

In [ ]:
inconsist_players.to_csv('inconsist_'+ report_name)
inconsist_players

In [ ]:
extremly_inconsistant_players

In [ ]:
players_stat['Erin Burns']

In [221]:
p_df.to_csv(report_name)

In [168]:
'''p_df = pd.DataFrame(columns=['player', 'weight_points','one_match_per', 'total_points', 'coef_var', 'int_matches', 
                                             'int_points', 'bat_points', 'bowl_points','catch_points'])
p = players_stat
for i in players_stat:
    try:
        df = p[i][:3]
        coef = (df['total_points'].std() / df['total_points'].mean()) * 100
        weighted_points = cal_weighted_points(df['total_points'].mean(), coef)    
        one_match_per = (df['total_points'].max() / df['total_points'].sum()) * 100
        int_match_count = df['is_international'].sum()
        int_match_points = df[df['is_international']]['total_points'].sum() / (int_match_count + 0.001)
        row = pd.DataFrame({
                            'player': [i], 
                            'weight_points': [weighted_points],
                            'one_match_per': [one_match_per],
                            'total_points': [df['total_points'].mean()],
                            'coef_var': [coef],
                            'int_matches':[int_match_count],
                            'int_points': [int_match_points],
                            'bat_points': [df['bat_points'].mean()],
                            'bowl_points': [df['bowl_points'].mean()],
                            'catch_points': [df['catch_points'].mean()],
                           })
        p_df = p_df.append(row, ignore_index=True)
    except:
        print(df)
p_df = p_df.sort_values(by=['weight_points'], ascending=False)'''

"p_df = pd.DataFrame(columns=['player', 'weight_points','one_match_per', 'total_points', 'coef_var', 'int_matches', \n                                             'int_points', 'bat_points', 'bowl_points','catch_points'])\np = players_stat\nfor i in players_stat:\n    try:\n        df = p[i][:3]\n        coef = (df['total_points'].std() / df['total_points'].mean()) * 100\n        weighted_points = cal_weighted_points(df['total_points'].mean(), coef)    \n        one_match_per = (df['total_points'].max() / df['total_points'].sum()) * 100\n        int_match_count = df['is_international'].sum()\n        int_match_points = df[df['is_international']]['total_points'].sum() / (int_match_count + 0.001)\n        row = pd.DataFrame({\n                            'player': [i], \n                            'weight_points': [weighted_points],\n                            'one_match_per': [one_match_per],\n                            'total_points': [df['total_points'].mean()],\n                  

In [ ]:
'''stadium_url = 'http://stats.espncricinfo.com/westindies/engine/records/team/match_results.html?class=3;id=199;type=ground'
request=urllib2.Request(stadium_url,None,headers)
response = urllib2.urlopen(request)
data = response.read()
page_soup = soup(data, 'html.parser')

matches = page_soup.find('table', {'class': 'engineTable'}).find_all('td', {'nowrap': 'nowrap'})

match_list = []
for item in matches:
    match_link_html = item.find('a')
    if match_link_html:
        match_type = match_link_html.contents[0]
        if 'T20' in match_type:
            match_list.append('http://stats.espncricinfo.com' + match_link_html['href'])'''

In [ ]:
'''url = 'http://stats.espncricinfo.com/westindies/engine/match/1077947.html'
request=urllib2.Request(url,None,headers)
response = urllib2.urlopen(request)
data = response.read()
page_soup = soup(data, 'html.parser')

article = page_soup.find('article',{'class': 'sub-module scorecard'})

for match in match_list:
    time.sleep(2)
    request=urllib2.Request(match,None,headers)
    response = urllib2.urlopen(request)
    data = response.read()
    page_soup = soup(data, 'html.parser')
    #page_soup.body.findAll(text='Python Jobs') 
    for index, item in player_details.iterrows():
        if page_soup.body.findAll(text= item['link']):
            print(match)
            print(item['name'])'''